In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import webbrowser
import os
import csv
import pandas as pd

In [2]:
# Define files
base_file = "base_data.csv"
state_file = "state_data.csv"

In [3]:
# Open the CSV files

base_file_df = pd.read_csv(base_file, encoding="ISO-8859-1")
base_file_df

,Base
0,Ft Rucker
1,WAATS
2,EAATS
3,Ft Carson
4,Hunter AAF
5,Ft Riley
6,Ft Bragg
7,Ft Drum
8,Ft Bliss
9,Ft Hood


In [4]:
# Open the CSV files

state_file_df = pd.read_csv(state_file, encoding="ISO-8859-1")
state_file_df

,State
0,AL
1,AZ
2,PA
3,CO
4,GA
5,KS
6,NC
7,NY
8,TX
9,TX


In [5]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [6]:
# Define database and collection
db = client.nhl_db
collection = db.articles

In [14]:
Base_Locations_df = pd.DataFrame(columns=['Name', 'State', 'Latitude', 'Longitude'])
#request_headers = {
#    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
#    'accept-encoding': 'gzip, deflate, br',
#    'accept-language': 'en-US,en;q=0.8',
#    'upgrade-insecure-requests': '1',
#    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
#}

text = 'GPS coordinates of '
space = ' '
location_string = ' '

#count = base_file_df.shape[0]
count = 5
i = 0

while i < count:
    try:
        location_string = ' '
        base_name = str(base_file_df.loc[i].item())
        state = str(state_file_df.loc[i].item())
        url = 'http://google.com/search?q=' + text + base_name + space + state
        print(url)
        
        response = requests.get(url)
        #with requests.Session() as s:
        #    response = s.get(url, headers=request_headers)
        
        #print(response)
        soup = BeautifulSoup(response.text, 'lxml')
        for g in soup.find_all(class_='g'):
            location_string = g.text
            break
        
        location_string = location_string.split(':')
        print(location_string)
        base_lat = location_string[3].rsplit(' ')[1]
        base_long = location_string[4].rsplit(' ')[1]   

        print(f'Base:{base_name} {state}')
        print(f'Lat:{base_lat}')
        print(f'Long:{base_long}')
        Base_Locations_df = Base_Locations_df.append({'Name': base_name, 'State': state, 'Latitude': base_lat, 'Longitude': base_long}, ignore_index=True)
        i = i + 1
        
    except:
        print(response)
        i = i + 1


http://google.com/search?q=GPS coordinates of Ft Rucker AL
['GPS coordinates of Fort Rucker, United States. Latitude', ' 31.3428 ...https', '//latitude.to/map/us/united-states/cities/fort-ruckerCachedGPS coordinates of Fort Rucker, United States. Latitude', ' 31.3428 Longitude', ' -85.7154.']
Base:Ft Rucker AL
Lat:31.3428
Long:-85.7154.
http://google.com/search?q=GPS coordinates of WAATS AZ
['Arizona, USA Map Lat Long Coordinates - Latitude and Longitudehttps', "//www.latlong.net/place/arizona-usa-2688.htmlCachedThe latitude of Arizona, USA is 34.048927, and the longitude is -111.093735. Arizona, USA is located at United States country in the States place category with the gps coordinates of 34° 2' 56.1372'' N and 111° 5' 37.4460'' W."]
<Response [200]>
http://google.com/search?q=GPS coordinates of EAATS PA
["Pennsylvania, USA Lat Long Coordinates Info. The latitude of Pennsylvania, USA is 41.203323, and the longitude is -77.194527. Pennsylvania, USA is located at United States country

In [15]:
Base_Locations_df
   

   


,Name,State,Latitude,Longitude
0,Ft Rucker,AL,31.3428,-85.7154.
1,Ft Carson,CO,38.5533,-104.8388.


In [ ]:
# Dictionary to be inserted into MongoDB
    base_location = {
        'base': base_name,
        'state': state,
        'latitude': base_lat,
        'longitude': base_long
    }

In [ ]:
 # Insert dictionary into MongoDB as a document
    collection.insert_one(base_location)